Open and reshape data

In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from os import listdir
import pdb


In [2]:
# READ AND FORMAT DATA FILES

where='/home/ahaimovici/Documents/Teaching/Rectangle_Game/Data/Dynamic/with_time_info/';
files=[];
for file in listdir(where):
    if file.endswith(".csv"):
        files.append(file);

        
        
        #def read_files(where,files):   

Data=[];
for f in files:
    print(f)
    Pair={};
    i1=f.find('-');
    i2=f.find('_');
    i3=f[i1+1:].find('-')+i1+1;
    i4=f[i2+1:].find('_')+i2+1;
    Pair['Teachers_age']=int(f[0:i1]);
    Pair['Teachers_Id']=int(f[i1+1:i2]);
    Pair['Learners_age']=int(f[i2+1:i3]);    
    Pair['Learners_Id']=int(f[i3+1:i4]);             
    if Pair['Teachers_Id']>=400 or Pair['Learners_Id']>=400:
        Pair['Conscent']=False;
    else:
        Pair['Conscent']=True;

    df = pd.read_csv(where+f, delimiter=',');
    columns=df.columns.tolist();        
    tuples = [tuple(x) for x in df.values];
    Pair['Total_time']=int(tuples[-1][columns.index('time_elapsed')])/60000; # minutes
    Pair['Trials']=[];
    n_trial=0;
    for row in tuples:
        if row[1]=='teacher' or row[1]=='learner':
            n_trial+=1;
            if n_trial>2: # first two trials are training                  
                examples=row[columns.index('Examples')];
                inside=row[columns.index('Inside')];
                inside=inside.split(',');
                examples_list=[];                
                if not(pd.isnull(examples)):
                    examples=examples.split(",");            
                    for element in examples:                
                        if element[0]=='[':
                            element=element[1:];
                        if element[-1]==']':
                            element=element[0:-1];
                        examples_list.append(float(element));
                    examples_list = list(zip(*[iter(examples_list)] * 2));
                Examples=[];                
                i=0;
                for element in examples_list:
                    Examples.append({'x':element[0],'y':element[1],'inside':inside[i]})
                    i+=1;
                
                
                
                teachers_box=row[columns.index('Teachers_Box')];
                teachers_box=teachers_box[1:-1];
                teachers_box=teachers_box.split(",");   
                aux=[];
                for element in teachers_box:
                    element=int(element);
                    aux.append(element);
                teachers_box=aux;
                learners_guess=row[columns.index('Learners_Guess')];
                learners_guess=learners_guess.split(",");
                learners_guess_list=[];
                for element in learners_guess:
                    if element[0]=='[':
                        element=element[1:];
                    if element[-1]==']':
                        element=element[0:-1];
                    learners_guess_list.append(float(element));
                learners_guess_list = list(zip(*[iter(learners_guess_list)] * 4))  
                won=row[columns.index('won')];
                Pair['Trials'].append({'Teachers_box':teachers_box,'Examples':Examples,'n_examples':len(Examples),'Learners_guess':learners_guess_list,'won':won});
    Data.append(Pair);
#    return Data;

10-436_13-437_learner_dynamic.csv
13-437_10-436_teacher_dynamic.csv
10-305_13-434_teacher_dynamic.csv
28-63_13-304_learner_dynamic.csv
28-63_13-304_teacher_dynamic.csv
13-437_10-436_learner_dynamic.csv
10-305_13-434_learner_dynamic.csv
10-436_13-437_teacher_dynamic.csv


In [5]:
Data

[{'Conscent': False,
  'Learners_Id': 437,
  'Learners_age': 13,
  'Teachers_Id': 436,
  'Teachers_age': 10,
  'Total_time': 13.67405,
  'Trials': [{'Examples': [{'inside': 'true', 'x': 368.0, 'y': 386.0},
     {'inside': 'true', 'x': 366.0, 'y': 630.0},
     {'inside': 'true', 'x': 305.0, 'y': 502.0},
     {'inside': 'true', 'x': 433.0, 'y': 500.0}],
    'Learners_guess': [(363.0, 861.0, 382.0, 862.0),
     (362.0, 865.0, 380.0, 636.0),
     (300.0, 862.0, 377.0, 640.0),
     (297.0, 442.0, 375.0, 639.0)],
    'Teachers_box': [301, 435, 380, 632],
    'n_examples': 4,
    'won': True},
   {'Examples': [{'inside': 'false', 'x': 771.0, 'y': 521.0},
     {'inside': 'false', 'x': 484.0, 'y': 607.0},
     {'inside': 'false', 'x': 473.0, 'y': 519.0},
     {'inside': 'false', 'x': 773.0, 'y': 611.0}],
    'Learners_guess': [(7.0, 766.0, 8.0, 515.0),
     (5.0, 761.0, 5.0, 598.0),
     (480.0, 765.0, 3.0, 513.0),
     (481.0, 767.0, 528.0, 606.0)],
    'Teachers_box': [476, 757, 521, 607],
  

In [4]:
n_trials=[];
n_examples=[];
sd_examples=[];
n_examples_win=[];
sd_examples_win=[];
p_win=[];
time=[];
edades=[];
workable_trial=0;
non_workable_trial=0;
workable_couple=0;
for P in Data:
    edades.append(str(P['Teachers_age'])+'-'+str(P['Learners_age']))
    n_trials.append(len(P['Trials']));    
    time.append(P['Total_time']);    
    nex=[];
    nex_w=[];
    n_won=0;
    for trial in P['Trials']:
        nex.append(len(trial['Circles'])+len(trial['Crosses']));
        if trial['won']:
            nex_w.append(len(trial['Circles'])+len(trial['Crosses']));            
    n_examples.append(np.mean(nex));
    sd_examples.append(np.std(nex));
    n_examples_win.append(np.mean(nex_w));
    sd_examples_win.append(np.std(nex_w));
    p_win.append(len(nex_w)/len(nex));
    flag=True;
    if not(P['Conscent']):
        print(P['Teachers_Id'])
    for trial in P['Trials']:
        if not trial['Crosses'] or not trial['Circles']:            
            workable_trial+=1;
            if flag:
                workable_couple+=1;
                flag=False;
                

KeyError: 'Circles'

In [ ]:
def individual_histogram(pair,center): # as in the paper. # center: make the box be in the center of the crosses histogram
    n_bins=3;
    canvas_size=865;
    inner_bins=np.zeros([n_bins,n_bins]); 
    if center:
        outer_bins=np.zeros([2*n_bins+1,2*n_bins+1]);             
    else:
        outer_bins=np.zeros([n_bins+1,n_bins+1]);             
    n_trial=-1;
    for trial in pair['Trials']:
            n_trial=n_trial+1;
            box=trial['Teachers_box'];
            width=box[1]-box[0];
            height=box[3]-box[2];                
            for circle in trial['Circles']:
                x=circle[0]-box[0];
                y=circle[1]-box[2];
                x_bin=np.int(np.floor(x/(width/n_bins)));
                y_bin=np.int(np.floor(y/(height/n_bins)));
                if(x_bin==3):
                    x_bin=2;
                if(y_bin==3):
                    y_bin=2;                
                inner_bins[x_bin,y_bin ]=inner_bins[x_bin,y_bin]+1;                
            for cross in trial['Crosses']:
                if (cross[0]-box[0])<0: #left side
                    x=box[0]-cross[0];
                    x_bin=np.int(np.floor(x/(box[0]/n_bins)))+1; # bins 1,2 or 3
                    if center:
                        x_bin=-x_bin+n_bins; # make the box be in the center
                elif cross[0]>box[1]: #right side
                    x=cross[0]-box[1];
                    x_bin=np.int(np.floor(x/((canvas_size-box[1])/n_bins)))+1;
                    if center:
                        x_bin=x_bin+n_bins;
                else: # above or below the box
                    x_bin=0;
                    if center:
                        x_bin=x_bin+n_bins;
                if (cross[1]-box[2])<0: # above
                    y=box[2]-cross[1];
                    y_bin=np.int(np.floor(y/(box[2]/n_bins)))+1;
                    if center:
                        y_bin=y_bin+n_bins;
                elif cross[1]>box[3]: # below
                    y=cross[1]-box[3];
                    y_bin=np.int(np.floor(y/((canvas_size-box[3])/n_bins)))+1;
                    if center:
                        y_bin=-y_bin+n_bins;
                else:                                                                              
                    y_bin=0;                
                    if center:
                        y_bin=y_bin+n_bins;
                outer_bins[x_bin,y_bin]=outer_bins[x_bin,y_bin]+1;            
    return inner_bins, outer_bins;


def plot_inner_histo(histo_inside):
    fig=plt.figure();
    plt.pcolor(np.flip(histo_inside,0),cmap='Greys')     
    plt.colorbar();    
    plt.plot([0,3,3,0,0],[0,0,3,3,0],'b')
    plt.xlim([-0.5,3.5])
    plt.ylim([-0.5,3.5])
    plt.tick_params(bottom=False,top=False,left=False,right=False,labelbottom=False,labelleft=False)    
    plt.show()
    
    
    
def plot_outer_histo(histo_outside,center):
    fig=plt.figure();
    plt.title('')        
    if center:
        plt.pcolor(np.flip(histo_outside,0),cmap='Greys')
        plt.plot([3,4,4,3,3],[3,3,4,4,3],'b')
        plt.xlim([0,7])
        plt.ylim([0,7])    
    else:
        plt.pcolor(histo_outside,cmap='Greys')
        plt.plot([0,1,1,0,0],[0,0,1,1,0],'b')
        plt.xlim([-0.03,4])
        plt.ylim([-0.01,4])    
    plt.colorbar();    
    plt.tick_params(bottom=False,top=False,left=False,right=False,labelbottom=False,labelleft=False)
    plt.show()

In [ ]:
i=0;
for pair in Data:    
    [inner,outer]=individual_histogram(pair,True);
    print('Edad-T:'+str(pair['Teachers_age'])+' Edad-L:'+str(pair['Learners_age']))
    plot_inner_histo(inner)
    plot_outer_histo(outer,True)
    

In [ ]:
print(inner)
print(np.flip(inner,0))